Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "08")
cd(ProjDir)

### snippet 5.1

In [3]:
d = CSV.read(rel_path("..", "data", "rugged.csv"), delim=';');
df = convert(DataFrame, d);

dcc = filter(row -> !(ismissing(row[:rgdppc_2000])), df)
dcc[:log_gdp] = log.(dcc[:rgdppc_2000])
dcc[:cont_africa] = Array{Float64}(convert(Array{Int}, dcc[:cont_africa]))
first(dcc[[:rugged, :cont_africa, :log_gdp]], 5)

,rugged,cont_africa,log_gdp
,Float64⍰,Float64,Float64
1,0.858,1.0,7.49261
2,3.427,0.0,8.21693
3,0.769,0.0,9.93326
4,0.775,0.0,9.40703
5,2.688,0.0,7.79234


Define the Stan language model

In [4]:
m_8_1_model = "
data{
    int N;
    vector[N] log_gdp;
    vector[N] cont_africa;
    vector[N] rugged;
}
parameters{
    real a;
    real bR;
    real bA;
    real bAR;
    real sigma;
}
model{
    vector[N] mu;
    sigma ~ uniform( 0 , 10 );
    bAR ~ normal( 0 , 10 );
    bA ~ normal( 0 , 10 );
    bR ~ normal( 0 , 10 );
    a ~ normal( 0 , 100 );
    for ( i in 1:N ) {
        mu[i] = a + bR * rugged[i] + bA * cont_africa[i] + bAR * rugged[i] * cont_africa[i];
    }
    log_gdp ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [5]:
stanmodel = Stanmodel(name="m_8_1_model",
monitors = ["a", "bR", "bA", "bAR", "sigma"],
model=m_8_1_model, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/08/tmp/m_8_1_model.stan will be updated.



Input data for cmdstan

In [6]:
m_8_1_data = Dict("N" => size(dcc, 1), "log_gdp" => dcc[:log_gdp],
    "rugged" => dcc[:rugged], "cont_africa" => dcc[:cont_africa]);
# Sample using cmdstan
rc, chn, cnames = stan(stanmodel, m_8_1_data, ProjDir, diagnostics=false,
  summary=true, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
       mean   sd  5.5% 94.5% n_eff Rhat
a      9.22 0.14  9.00  9.46   282    1
bR    -0.21 0.08 -0.33 -0.08   275    1
bA    -1.94 0.24 -2.33 -1.59   268    1
bAR    0.40 0.14  0.18  0.62   271    1
sigma  0.96 0.05  0.87  1.04   339    1
"

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -0.190999, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/08/tmp/m_8_1_model.stan' at line 24)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -1.40202, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/08/tmp/m_8_1_model.stan' at line 24)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severe

"\n       mean   sd  5.5% 94.5% n_eff Rhat\na      9.22 0.14  9.00  9.46   282    1\nbR    -0.21 0.08 -0.33 -0.08   275    1\nbA    -1.94 0.24 -2.33 -1.59   268    1\nbAR    0.40 0.14  0.18  0.62   271    1\nsigma  0.96 0.05  0.87  1.04   339    1\n"

Describe the draws

In [7]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
         Mean         SD        Naive SE       MCSE      ESS
    a  9.2269937 0.140898924 0.00222780759 0.0029030513 1000
   bR -0.2046551 0.077269871 0.00122174393 0.0015543556 1000
   bA -1.9505869 0.222299750 0.00351486766 0.0052752597 1000
  bAR  0.3947156 0.130352993 0.00206106179 0.0027812662 1000
sigma  0.9521969 0.053849608 0.00085143706 0.0010395082 1000

Quantiles:
          2.5%       25.0%      50.0%      75.0%        97.5%   
    a  8.94205725  9.1338375  9.2295100  9.32274500  9.489816750
   bR -0.35598968 -0.2568240 -0.2044945 -0.15242100 -0.048401905
   bA -2.38136675 -2.1005900 -1.9520650 -1.80190250 -1.510729250
  bAR  0.13656818  0.3074065  0.3977695  0.48104450  0.643824425
sigma  0.85529272  0.9137510  0.9498675  0.98659275  1.065015750



End of `08/m8.1s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*